#### Objectives

We want to find a partner user who has trained, onboarded AND done a deal; and look at their login frequency 

Then we want to consider when the user was created, when they completed a training, completed an onboarding, and how many times they logged in before and they trained vs. after. 

#### Notes

New Partner Plan Task Completed between Sep 2022 and March 2023.

This will allow us to see trends in login frequency for 6 - 12 months after they completed training.

Filter - Plan Task = any of the three tasks in Milestone 3

After cleaning, Date column is the last date they completed any of the three tasks (if they completed more than one)

## Data Cleaning and EDA

In [1]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

In [2]:
import pathlib
DATA_DIR = pathlib.Path('data/').absolute()
RESULTS_DIR = pathlib.Path('results/').absolute()
SOURCE_DIR = pathlib.Path('data/00-source').absolute()
CLEANED_DIR = pathlib.Path('data/01-cleaned').absolute()
PROCESSED_DIR = pathlib.Path('data/02-processed').absolute()

assert DATA_DIR.exists(), f'Couldn\t find {DATA_DIR}. Check your path'
assert RESULTS_DIR.exists(), f'Couldn\t find {RESULTS_DIR} folder. Check your path'
assert SOURCE_DIR.exists(), f'Couldn\t find {SOURCE_DIR}. Check your path'
assert CLEANED_DIR.exists(), f'Couldn\t find {CLEANED_DIR} folder. Check your path'
assert PROCESSED_DIR.exists(), f'Couldn\t find {PROCESSED_DIR}. Check your path'

In [3]:
onboarded_m3 = pd.read_csv(SOURCE_DIR/'new_partner_m3.csv')
login_data = pd.read_csv(SOURCE_DIR/'login.csv')

In [4]:
onboarded_m3.columns = ['Client','Plan','Onboarded Month','Name','Task'] # Change column names
onboarded_m3['Onboarded Month'] = pd.to_datetime(onboarded_m3['Onboarded Month']) # Convert onboarded data Date column to datetime object

In [5]:
idx = onboarded_m3.groupby('Name')['Onboarded Month'].idxmax() # Get the latest date the user complete any of the three tasks in milestone 3
onboarded_m3_latest = onboarded_m3.loc[idx]

In [ ]:
onboarded_m3_latest

## Onboarded User Login Trends

In [11]:
login_data['Partner User Full Name'].nunique()

1338

In [7]:
onboarded_users = onboarded_m3_latest['Name'] # Get the list of users onboarded between Jan and Nov 2022.

In [ ]:
onboarded_user_login = login_data[login_data['Partner User Full Name'].isin(onboarded_users)] # Filter login data, get the log in info ONLY for onboarded users
onboarded_user_login

In [9]:
onboarded_user_login['Partner User Full Name'].nunique()

231

In [9]:
onboarded_user_login['Dates (Login) Month with Year'] = pd.to_datetime(onboarded_user_login['Dates (Login) Month with Year']) # Conveert Date column to datetime object
onboarded_user_login.columns = ['Month','Name','Logins','TPL Logins'] # Change column names

/var/folders/cs/_ws68z551mdg6_6_dmsht4tc0000gq/T/ipykernel_62949/1086635909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onboarded_user_login['Dates (Login) Month with Year'] = pd.to_datetime(onboarded_user_login['Dates (Login) Month with Year']) # Conveert Date column to datetime object


In [10]:
onboarded_m3_latest.columns

Index(['Client', 'Plan', 'Onboarded Month', 'Name', 'Task'], dtype='object')

In [11]:
onboarded_user_login.columns

Index(['Month', 'Name', 'Logins', 'TPL Logins'], dtype='object')

In [12]:
merged_df = onboarded_user_login.merge(onboarded_m3_latest[['Name', 'Onboarded Month']], 
                                       on='Name', 
                                       how='left')

In [ ]:
merged_df['Months After Onboarding'] = ((merged_df['Month'] - merged_df['Onboarded Month']).dt.days / 30.44).astype(int)
merged_df

In [14]:
merged_df['TPL Logins'] = merged_df['TPL Logins'].replace('-', 0)
merged_df['Logins'] = merged_df['Logins'].replace('-', 0)
merged_df['TPL Logins'] = merged_df['TPL Logins'].astype('int')
merged_df['Logins'] = merged_df['Logins'].astype('int')

#### Average Login by Months after Onboarding
##### On average, users logged in 5 times into the portal at the month they completed their last milestone 3 task. However, log in frequency starts diminishing at the second month after they onboarded.
##### Note that these numbers only accounts for users that STILL logged in to their portal after onboarding.

In [15]:
average_monthly_login = merged_df[['TPL Logins','Months After Onboarding']].groupby('Months After Onboarding').mean()
average_monthly_login = average_monthly_login.rename(columns={"TPL Logins": "Average Monthly Logins"})
average_monthly_login = average_monthly_login[average_monthly_login.index>=0]
average_monthly_login

,Average Monthly Logins
Months After Onboarding,
0,5.151625
1,5.450000
2,4.340909
3,3.166667
4,3.200000
5,3.833333
6,3.500000
7,2.600000
8,2.000000


In [16]:
fig = px.line(average_monthly_login, 
              x=average_monthly_login.index, 
              y='Average Monthly Logins', 
              labels={'index': 'Months After Onboarding', 'Average Monthly Logins': 'Average Monthly Logins'},
              title='Average Monthly Logins vs. Months After Onboarding')
fig.update_layout(width = 800, height = 500)

fig.show()

### Count Users who Logged In by Months after Onboarding
##### 277 users logged in to their portal at the month they completed their last milestone 3 task. Only 2 of them logged in to their portal 11 months after onboarding.

In [17]:
average_monthly_users = merged_df[['Months After Onboarding','Name']].groupby('Months After Onboarding').count()
average_monthly_users = average_monthly_users.rename(columns = {"Name":"Count Users"})
average_monthly_users = average_monthly_users[average_monthly_users.index>=0]
average_monthly_users

,Count Users
Months After Onboarding,
0,277
1,40
2,44
3,24
4,20
5,12
6,10
7,20
8,6


In [18]:
fig = px.line(average_monthly_users, 
              x=average_monthly_users.index, 
              y='Count Users', 
              labels={'index': 'Months After Onboarding', 'Count Users': 'Count Users'},
              title='Count Users who Logged In vs. Months After Onboarding')
fig.update_layout(width = 800, height = 500)

fig.show()

In [33]:
average_monthly_users['Percentage Change'] = average_monthly_users['Count Users'].pct_change() * 100  # multiply by 100 to get the value in percentage

# Graph the percentage change using Plotly
fig = go.Figure(data=[go.Bar(x=average_monthly_users.index, y=average_monthly_users['Percentage Change'])])
fig.update_layout(title='Percentage Change in Number of Logged-In Users Over Time', xaxis_title='Months after Onboarding', yaxis_title='Percentage Change', width = 800, height = 500)
fig.show()

## Unonboarded User Login Trends

### Overall average monthly login among unonboarded users

In [ ]:
unonboarded_user_login = login_data[~login_data['Partner User Full Name'].isin(onboarded_users)] # Filter login data, get the log in info for unonboarded users
unonboarded_user_login

In [20]:
unonboarded_user_login['Dates (Login) Month with Year'] = pd.to_datetime(unonboarded_user_login['Dates (Login) Month with Year']) # Conveert Date column to datetime object
unonboarded_user_login.columns = ['Month','Name','Logins','TPL Logins'] # Change column names

/var/folders/cs/_ws68z551mdg6_6_dmsht4tc0000gq/T/ipykernel_62949/2209164977.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
def compute_month_number(date):
    if date.year == 2022:
        return date.month - 9
    else:
        return date.month + 3

unonboarded_user_login['Month Number'] = unonboarded_user_login['Month'].apply(compute_month_number)
unonboarded_user_login = unonboarded_user_login[unonboarded_user_login['Month Number'].notna()]

/var/folders/cs/_ws68z551mdg6_6_dmsht4tc0000gq/T/ipykernel_62949/2370232447.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:
unonboarded_user_login = unonboarded_user_login[unonboarded_user_login['TPL Logins']!='-']
unonboarded_user_login['TPL Logins'] = unonboarded_user_login['TPL Logins'].astype(int)

In [23]:
unonboarded_user_login['TPL Logins'].describe()

count    2126.000000
mean        3.341486
std         4.643431
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max        60.000000
Name: TPL Logins, dtype: float64

In [24]:
unonboarded_user_login['TPL Logins'].median()

2.0

Average monthly login among unonboarded users = 3.3\
Median monthly login among unonboarded users = 2

### Unonboarded User Login by Month

In [25]:
unonboarded_monthly_login = unonboarded_user_login.groupby(['Month'])[['TPL Logins','Month Number']].mean().reset_index()
unonboarded_monthly_login

,Month,TPL Logins,Month Number
0,2022-09-01,2.302158,0.0
1,2022-10-01,2.113208,1.0
2,2022-11-01,2.076923,2.0
3,2022-12-01,2.227273,3.0
4,2023-01-01,2.609375,4.0
5,2023-02-01,2.676923,5.0
6,2023-03-01,3.447917,6.0
7,2023-04-01,3.696429,7.0
8,2023-05-01,3.563725,8.0
9,2023-06-01,3.840164,9.0


In [26]:
fig = px.line(unonboarded_monthly_login, 
              x='Month', 
              y='TPL Logins', 
              labels={'Month': 'Month', 'Count Users': 'Count Users'},
              title='Login Frequency for Unonboarded Users')
fig.update_layout(width = 800, height = 500)

fig.show()

In [27]:
login_data.columns = ['Month','Name','Logins','TPL Logins']
login_data['Month'] = pd.to_datetime(login_data['Month'])

In [28]:
login_data = login_data[login_data['Month'].notna()]
login_data = login_data[login_data['TPL Logins']!='-']
login_data['TPL Logins'] = login_data['TPL Logins'].astype(int)

In [29]:
average_monthly_login_all = login_data[['Month','TPL Logins']].groupby(['Month']).mean().reset_index()
average_monthly_login_all

,Month,TPL Logins
0,2022-09-01,2.815920
1,2022-10-01,2.887500
2,2022-11-01,2.750000
3,2022-12-01,3.215190
4,2023-01-01,3.660194
5,2023-02-01,3.854167
6,2023-03-01,3.662281
7,2023-04-01,3.652406
8,2023-05-01,3.529148
9,2023-06-01,3.857708


In [30]:
fig = px.line(average_monthly_login_all, 
              x='Month', 
              y='TPL Logins', 
              labels={'Month': 'Month', 'TPL Logins': 'TPL Logins'},
              title='Log In Frequency for All Users')
fig.update_layout(width = 800, height = 500)

fig.show()